In [34]:
import warnings
 
# 모든 경고 메시지를 무시합니다.
warnings.filterwarnings("ignore")

In [9]:
import os
import json
import requests
import time
from openai import AzureOpenAI
from dotenv import load_dotenv

load_dotenv()

client = AzureOpenAI(
  azure_endpoint = os.getenv("EXER_AZURE_OPENAI_ENDPOINT"),
  api_key= os.getenv("EXER_AZURE_OPENAI_API_KEY"),
  api_version="2024-05-01-preview"
)


WEATHER_DATA = {
    "tokyo": None,
    "san francisco": None,
    "paris": None,
    "seoul": None
}

# Simplified timezone data
TIMEZONE_DATA = {
    "tokyo": "Asia/Tokyo",
    "san francisco": "America/Los_Angeles",
    "paris": "Europe/Paris",
    "seoul" : "Asia/Seoul"
}

def get_current_weather(location, unit=None):
    """Get the current weather for a given location"""
    location_lower = location.lower()

    print(f"get_current_weather called with location: {location}, unit: {unit}")  
    
    for key in WEATHER_DATA:
        if key in location_lower:
            print(f"Weather data found for {key}")  
            # weather = WEATHER_DATA[key]

            geo_url = f"https://geocoding-api.open-meteo.com/v1/search?name={key}&count=1&language=ko"
            geo = requests.get(geo_url).json()

            lat, lon = geo['results'][0]['latitude'], geo['results'][0]['longitude']

            weather_url = f"https://api.open-meteo.com/v1/forecast?latitude={lat}&longitude={lon}&current=temperature_2m,relative_humidity_2m,weather_code,rain,precipitation,wind_speed_10m,wind_direction_10m"
            data = requests.get(weather_url).json()

            return json.dumps({
                "location": location,
                "temperature": data["current"]["temperature_2m"],
                "relative_humidity": data["current"]["relative_humidity_2m"],
                "weather_code": data["current"]["weather_code"],
                "rain": data["current"]["rain"],
                "precipitation": data["current"]["precipitation"],
                "wind_speed": data["current"]["wind_speed_10m"],
                "wind_direction": data["current"]["wind_direction_10m"],
                "unit": None
            })
    
    print(f"No weather data found for {location_lower}")  
    return json.dumps({"location": location, "temperature": "unknown"})

def get_current_time(location):
    """Get the current time for a given location"""
    print(f"get_current_time called with location: {location}")  
    location_lower = location.lower()
    
    for key, timezone in TIMEZONE_DATA.items():
        if key in location_lower:
            print(f"Timezone found for {key}")  
            current_time = datetime.now(ZoneInfo(timezone)).strftime("%I:%M %p")
            return json.dumps({
                "location": location,
                "current_time": current_time
            })
    
    print(f"No timezone data found for {location_lower}")  
    return json.dumps({"location": location, "current_time": "unknown"})

assistant = client.beta.assistants.create(
  model="gpt-4o-mini", # replace with model deployment name.
  instructions="당신은 날씨를 알려주는 봇입니다. 모르는 지역은 솔직히 모른다고 하세요.",
  tools = [
      {
          "type": "function",
          "function": {
              "name": "get_current_weather",
              "description": "Get the current weather in a given location",
              "parameters": {
                  "type": "object",
                  "properties": {
                      "location": {
                          "type": "string",
                          "description": "The city name, e.g. San Francisco MUST be translated into English",
                      },
                      "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                  },
                  "required": ["location"],
              },
          }
      },
      {
          "type": "function",
          "function": {
              "name": "get_current_time",
              "description": "Get the current time in a given location",
              "parameters": {
                  "type": "object",
                  "properties": {
                      "location": {
                          "type": "string",
                          "description": "The city name, e.g. San Francisco MUST be translated into English",
                      },
                  },
                  "required": ["location"],
              },
          }
      }
  ],
  tool_resources={},
  temperature=1,
  top_p=1
)

In [ ]:
# Create a thread
thread = client.beta.threads.create()

# Add a user question to the thread
message = client.beta.threads.messages.create(
  thread_id=thread.id,
  role="user",
  content="서울이랑 파리의 날씨 알려줘"
)

# Run the thread
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id
)

# Looping until the run completes or fails
while run.status in ['queued', 'in_progress', 'cancelling']:
  
  time.sleep(1)
  run = client.beta.threads.runs.retrieve(
    thread_id=thread.id,
    run_id=run.id
  )

  if run.status == 'completed':
    messages = client.beta.threads.messages.list(
      thread_id=thread.id
    )
    # print(messages)
    print(messages.data[0].content[0].text.value)

  elif run.status == 'requires_action':

    tool_outputs = []

    for tool_call in run.required_action.submit_tool_outputs.tool_calls :
              
              function_name = tool_call.function.name
              function_args = json.loads(tool_call.function.arguments)
              print(f"Function call: {function_name}")  
              print(f"Function arguments: {function_args}")  
              
              if function_name == "get_current_weather" :
                  output = get_current_weather(
                      location=function_args.get("location"),
                      unit=function_args.get("unit")
                  )
                  tool_outputs.append({"tool_call_id": tool_call.id, "output": output})
              elif function_name == "get_current_time":
                  function_response = get_current_time(
                      location=function_args.get("location")
                  )
                  tool_outputs.append({"tool_call_id": tool_call.id, "output": output})
              else:
                  function_response = json.dumps({"error": "Unknown function"})
              
    run = client.beta.threads.runs.submit_tool_outputs(
        thread_id=thread.id,
        run_id=run.id,
        tool_outputs=tool_outputs
    )


Function call: get_current_weather
Function arguments: {'location': 'Seoul', 'unit': 'celsius'}
get_current_weather called with location: Seoul, unit: celsius
Weather data found for seoul
Function call: get_current_weather
Function arguments: {'location': 'Paris', 'unit': 'celsius'}
get_current_weather called with location: Paris, unit: celsius
Weather data found for paris
현재 서울의 날씨는 다음과 같습니다:
- 기온: 4.8°C
- 습도: 33%
- 바람 속도: 4.7 m/s
- 바람 방향: 북서쪽 (293°)

파리의 날씨는 다음과 같습니다:
- 기온: 8.7°C
- 습도: 92%
- 바람 속도: 8.0 m/s
- 바람 방향: 남쪽 (190°)

추운 날씨에 조심하세요!


In [ ]:
Run(id='run_Cjx6OqXSzlJucZssEhY45OoN',
    assistant_id='asst_k8bjMc9JpSmlUQZTJ5jiGkir',
    cancelled_at=None, completed_at=None,
    created_at=1764310251, expires_at=1764346251,
    failed_at=None, incomplete_details=None,
    instructions='당신은 날씨를 알려주는 봇입니다. 모르는 지역은 솔직히 모른다고 하세요.',
    last_error=None, max_completion_tokens=None, max_prompt_tokens=None,
    metadata={}, model='gpt-4o-mini', object='thread.run', parallel_tool_calls=True,
    required_action=RequiredAction(
        submit_tool_outputs=RequiredActionSubmitToolOutputs(
            tool_calls=[RequiredActionFunctionToolCall(
                id='call_6jJ3btx4C5kEfGi9ekZCG7wz',
                function=Function(arguments='{"location":"Seoul"}',
                                  name='get_current_weather'), type='function')]),
                                  type='submit_tool_outputs'), response_format='auto',
                                  started_at=1764310251, status='requires_action', thread_id='thread_HMUSGWhmF6XxTdxyViqAUjkW', tool_choice='auto',
                                  tools=[FunctionTool(function=FunctionDefinition(name='get_current_weather', description='Get the current weather in a given location', parameters={'type': 'object', 'properties': {'location': {'type': 'string', 'description': 'The city name, e.g. San Francisco MUST be translated into English'}, 'unit': {'type': 'string', 'enum': ['celsius', 'fahrenheit']}}, 'required': ['location']}, strict=False), type='function'), FunctionTool(function=FunctionDefinition(name='get_current_time', description='Get the current time in a given location', parameters={'type': 'object', 'properties': {'location': {'type': 'string', 'description': 'The city name, e.g. San Francisco MUST be translated into English'}}, 'required': ['location']}, strict=False), type='function')], truncation_strategy=TruncationStrategy(type='auto', last_messages=None), usage=None, temperature=1.0, top_p=1.0, tool_resources={})

In [ ]:
SyncCursorPage[Message](data=[Message(id='msg_dR6PnfvqR3QtIgc7Mov1n9GI',
                                      assistant_id='asst_k8bjMc9JpSmlUQZTJ5jiGkir', attachments=[],
                                      completed_at=None,
                                      content=[TextContentBlock(text=Text(annotations=[],
                                                                          value='현재 서울의 날씨는 다음과 같습니다:\n- 기온: 4.9°C\n- 상대 습도: 32%\n- 바람 속도: 4.7 m/s (서북서 방향)\n- 강수량: 0.0 mm\n\n추운 날씨이니 외출 시 따뜻하게 입으세요!'), type='text')], created_at=1764313051, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='assistant', run_id='run_DliJPqKixf3bCf3NsYj5mfRY', status=None, thread_id='thread_JuyA5u1qTObhvdjORDGQJtHs'), Message(id='msg_ggklzL6ikpfvmVVQHmTeIRk8', assistant_id=None, attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='서울 날씨 알려줘'), type='text')], created_at=1764313046, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_JuyA5u1qTObhvdjORDGQJtHs')], has_more=False, object='list', first_id='msg_dR6PnfvqR3QtIgc7Mov1n9GI', last_id='msg_ggklzL6ikpfvmVVQHmTeIRk8')
completed